### Prompt

In [1]:
import bs4, tiktoken, numpy as np, os

from langchain import hub
from langchain.load import dumps, loads
from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings

from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader

from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

from operator import itemgetter

from dotenv import load_dotenv
load_dotenv()

learn_api_key = os.environ['LANGCHAIN_API_KEY']
openai_api_key = os.environ['OPENAI_API_KEY']
groq_api_key = os.environ['GROQ_API_KEY']

In [2]:
### INDEXING ###
loader = WebBaseLoader(
    web_path=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

blog_docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=300, chunk_overlap=50)

# Make Splits
splits = text_splitter.split_documents(blog_docs)

# Index
vectorstore = Chroma.from_documents(
    splits,
    HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
)
retriever = vectorstore.as_retriever()

In [3]:
# HyDE document generation
template = """Please write a scientific paper passage to answer the question
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(template)

llm = ChatGroq(model='openai/gpt-oss-20b', api_key=groq_api_key, temperature=0)

generate_docs_for_retrieval = (prompt_hyde | ChatGroq(model="gemma2-9b-it", api_key=groq_api_key, temperature=0) | StrOutputParser())

question = "What is task decomposition for LLM agents?"
generate_docs_for_retrieval.invoke({"question": question})

'## Task Decomposition for Large Language Model Agents: Breaking Down Complexity for Effective Action\n\nLarge Language Model (LLM) agents, while demonstrating impressive capabilities, often struggle with complex tasks requiring multiple sequential steps or diverse sub-tasks. This limitation stems from their inherent nature as text-based models, lacking the explicit reasoning and planning mechanisms found in traditional AI agents. To address this, **task decomposition** emerges as a crucial technique, enabling LLMs to tackle complex tasks by breaking them down into smaller, more manageable sub-tasks.\n\nTask decomposition involves identifying the constituent sub-tasks within a larger goal and formulating them as individual, well-defined problems. This process can be achieved through various methods, including:\n\n* **Rule-based decomposition:** Utilizing predefined rules or templates to identify recurring patterns and decompose tasks accordingly.\n* **Hierarchical decomposition:** Brea

In [4]:
# Retrieve
retrieval_chain = generate_docs_for_retrieval | retriever
retrieved_docs = retrieval_chain.invoke({"question":question})
retrieved_docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Component One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a

In [5]:
# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (prompt | llm | StrOutputParser())

final_rag_chain.invoke({"context":retrieved_docs, "question":question})

'**Task decomposition for LLM agents** is the process by which an autonomous agent powered by a large language model (LLM) splits a complex, high‑level goal into a sequence of smaller, more manageable sub‑tasks or sub‑goals. This decomposition is essential for the agent to:\n\n1. **Make the problem tractable** – By breaking a big problem into bite‑size pieces, the agent can reason about each piece individually, which is far easier than tackling the whole problem at once.  \n2. **Plan and execute** – Each sub‑goal can be turned into a concrete plan (e.g., a list of steps or API calls) that the agent can follow.  \n3. **Reflect and refine** – After executing a sub‑goal, the agent can evaluate its success, learn from mistakes, and adjust subsequent sub‑goals accordingly.\n\n### How it’s done\n\n| Method | How it works | Typical prompt |\n|-------|-------------|---------------|\n| **LLM‑driven prompting** | The LLM itself is asked to enumerate the steps needed to achieve the overall goal. 